## Welcome to the Second Lab - Week 1, Day 3

Today we will work with lots of models! This is a way to get comfortable with APIs.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Important point - please read</h2>
            <span style="color:#ff7800;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you carefully execute this yourself, <b>after</b> watching the lecture. Add print statements to understand what's going on, and then come up with your own variations.<br/><br/>If you have time, I'd love it if you submit a PR for changes in the community_contributions folder - instructions in the resources. Also, if you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...
            </span>
        </td>
    </tr>
</table>

In [17]:
# Start with imports - ask ChatGPT to explain any package that you don't know

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display
from limits import parse
from limits.storage import RedisStorage
from limits.strategies import FixedWindowRateLimiter

In [18]:
# Always remember to do this!
load_dotenv(override=True)
storage = RedisStorage("redis://localhost:6379")

In [ ]:
MODEL_LIMITS = {
    # --- Gemini 2.5 Series ---
    "gemini-2.5-pro": {"rpd": 50, "rpm": 2, "tpm": 125000, "tpd":3000000},
    # "gemini-2.5-pro-1p-freebie": {"rpd": 500, "rpm": 750, "tpm": 1000000},
    "gemini-2.5-flash": {"rpd": 250, "rpm": 10, "tpm": 250000, "tpd":None},
    "gemini-2.5-flash-lite": {"rpd": 1000, "rpm": 15, "tpm": 250000, "tpd":None}, # can
    "gemini-2.5-flash-live": {"rpd": None, "rpm": None, "tpm": 1000000, "tpd":None},
    # "gemini-2.5-flash-tts": {"rpd": 150, "rpm": 30, "tpm": 10000},
    # "gemini-2.5-flash-native-audio-dialog": {"rpd": 50, "rpm": None, "tpm": 25000},
    # "gemini-2.5-flash-exp-native-audio-thinking-dialog": {"rpd": 50, "rpm": None, "tpm": 10000},

    # # --- Gemini 2.0 Series ---
    # "gemini-2.0-flash": {"rpd": 200, "rpm": 150, "tpm": 1000000},
    # "gemini-2.0-flash-lite": {"rpd": 2000, "rpm": 300, "tpm": 1000000},
    # "gemini-2.0-flash-live": {"rpd": None, "rpm": None, "tpm": 1000000},
    # "gemini-2.0-exp": {"rpd": 500, "rpm": 50, "tpm": 1000000},

    # # --- Gemini 1.5 Series ---
    # "gemini-1.5-flash": {"rpd": 500, "rpm": 150, "tpm": 250000},
    # "gemini-1.5-flash-8b": {"rpd": 500, "rpm": 150, "tpm": 250000},

    # # --- Gemma 3 Series ---
    # "gemma-3-1b": {"rpd": 14400, "rpm": 300, "tpm": 15000},
    # "gemma-3-2b": {"rpd": 14400, "rpm": 300, "tpm": 15000},
    # "gemma-3-4b": {"rpd": 14400, "rpm": 300, "tpm": 15000},
    # "gemma-3-12b": {"rpd": 14400, "rpm": 300, "tpm": 15000},
    # "gemma-3-27b": {"rpd": 14400,"rpm": 300,"tpm": 15000},
    # # --- Other & Experimental Models ---
    # "chat-bard": {"rpd": 2520000, "rpm": 18000, "tpm": None},
    # "computer-use-exp": {"rpd": None, "rpm": 1000, "tpm": None},
    # "gqi-cst-h34jgm": {"rpd": None, "rpm": 500, "tpm": 3000000},
    # "gemini-1.0-pro": {"rpd": 0, "rpm": 0, "tpm": 0},
    # "gemini-1.5-pro": {"rpd": 0, "rpm": 0, "tpm": 0},
    # "gemini-1.5-pro-exp": {"rpd": 0, "rpm": 0, "tpm": 0},
    # "gemini-1.5-flash-exp": {"rpd": 0, "rpm": 0, "tpm": 0},
    # "gemini-1.5-flash-8b-exp": {"rpd": 0, "rpm": 0, "tpm": 0},
    # "gemini-2.0-pro-exp": {"rpd": 0, "rpm": 0, "tpm": 0},
    # "gemini-2.5-pro-exp": {"rpd": 0, "rpm": 0, "tpm": 0},
    # "gemini-2.5-pro-tts": {"rpd": 0, "rpm": 0, "tpm": 0},
    # "gemini-2.5-flash-preview-image": {"rpd": 0, "rpm": 0, "tpm": 0},
}

In [20]:
# Print the key prefixes to help with any debugging

# openai_api_key = os.getenv('OPENAI_API_KEY')
# anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GEMINI_API_KEY')
# deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
# groq_api_key = os.getenv('GROQ_API_KEY')

# if openai_api_key:
#     print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
# else:
#     print("OpenAI API Key not set")
    
# if anthropic_api_key:
#     print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
# else:
#     print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

# if deepseek_api_key:
#     print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
# else:
#     print("DeepSeek API Key not set (and this is optional)")

# if groq_api_key:
#     print(f"Groq API Key exists and begins {groq_api_key[:4]}")
# else:
#     print("Groq API Key not set (and this is optional)")

Google API Key exists and begins AI


In [40]:
# def callLLmApi(api_key,base_url,model,messages):
#     llm = OpenAI(api_key=api_key, base_url=base_url)
#     model_name = model

#     response = llm.chat.completions.create(model=model_name, messages=messages)
#     response_content = response.choices[0].message.content

#     display(Markdown(response_content))
#     return response_content
# Updated callLLmApi function with rate limiting
# Updated callLLmApi function with rpd, rpm, tpd, tpm support
def callLLmApi(api_key, base_url, model, messages, expected_prompt_tokens=None, expected_completion_tokens=None):
    def _estimate_tokens(msgs):
        try:
            txt = "".join(m.get("content", "") for m in msgs if isinstance(m, dict))
            return max(1, len(txt) // 4)  # rough 4 chars/token fallback
        except Exception:
            return 0

    limits = MODEL_LIMITS.get(model, {})
    limiter = FixedWindowRateLimiter(storage)

    rpm = limits.get("rpm")
    rpd = limits.get("rpd")
    tpm = limits.get("tpm")
    tpd = limits.get("tpd")

    # Keys per model
    req_key = f"api:req:{model}"
    tpd_key = f"api:tok:day:{model}"
    tpm_key = f"api:tok:month:{model}"

    # Time windows
    limit_rpm = parse(f"{rpm}/minute") if rpm else None
    limit_rpd = parse(f"{rpd}/day") if rpd else None
    limit_tpd = parse(f"{tpd}/day") if tpd else None
    limit_tpm = parse(f"{tpm}/month") if tpm else None

    # Predicted token spend (used for pre-check). Fallback to prompt-only estimate if no hints provided.
    if expected_prompt_tokens is not None or expected_completion_tokens is not None:
        expected_total_tokens = (expected_prompt_tokens or 0) + (expected_completion_tokens or 0)
    else:
        expected_total_tokens = _estimate_tokens(messages)

    # Pre-check: ensure this call would not exceed any limits
    if limit_rpm and not limiter.test(limit_rpm, req_key, 1):
        display(Markdown(f"⚠️ Rate limit hit: {rpm} rpm for {model}. Please wait."))
        return None
    if limit_rpd and not limiter.test(limit_rpd, req_key, 1):
        display(Markdown(f"⚠️ Daily request limit hit: {rpd} rpd for {model}."))
        return None
    if limit_tpd and expected_total_tokens and not limiter.test(limit_tpd, tpd_key, expected_total_tokens):
        display(Markdown(f"⚠️ Daily token limit would be exceeded for {model}."))
        return None
    if limit_tpm and expected_total_tokens and not limiter.test(limit_tpm, tpm_key, expected_total_tokens):
        display(Markdown(f"⚠️ Monthly token limit would be exceeded for {model}."))
        return None

    # Reserve request slots (avoids race with other callers)
    if limit_rpm and not limiter.hit(limit_rpm, req_key, 1):
        display(Markdown(f"⚠️ Rate limit hit: {rpm} rpm for {model}. Please wait."))
        return None
    if limit_rpd and not limiter.hit(limit_rpd, req_key, 1):
        display(Markdown(f"⚠️ Daily request limit hit: {rpd} rpd for {model}."))
        return None

    llm = OpenAI(api_key=api_key, base_url=base_url)
    model_name = model

    # Real call (uncomment for production)
    response = llm.chat.completions.create(model=model_name, messages=messages)
    response_content = response.choices[0].message.content

    # Try to capture actual token usage if provided by the API
    actual_total_tokens = None
    # try:
    #     usage = getattr(response, "usage", None)
    #     if isinstance(usage, dict):
    #         actual_total_tokens = usage.get("total_tokens")
    #     else:
    #         actual_total_tokens = getattr(usage, "total_tokens", None)
    # except Exception:
    #     actual_total_tokens = None

    # Token spend accounting (fallback to expected if actual unavailable)
    token_spend = actual_total_tokens if actual_total_tokens is not None else expected_total_tokens

    # Record token usage against daily/monthly quotas
    if token_spend and (limit_tpd or limit_tpm):
        if limit_tpd and not limiter.hit(limit_tpd, tpd_key, token_spend):
            display(Markdown(f"⚠️ Daily token limit exceeded for {model}."))
            return None
        if limit_tpm and not limiter.hit(limit_tpm, tpm_key, token_spend):
            display(Markdown(f"⚠️ Monthly token limit exceeded for {model}."))
            return None

    # Mocked response for now
    # response_content = "mock calling"
    display(Markdown(response_content))
    return response_content

In [42]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [46]:
result = callLLmApi(
    google_api_key,
    "https://generativelanguage.googleapis.com/v1beta/openai/",
    "gemini-2.5-flash-lite",
    messages
)

If you were tasked with creating a universal ethical framework for artificial general intelligence, what single, non-negotiable principle would you prioritize, and why would that principle be more fundamental than all others in ensuring beneficial AI development, even when faced with seemingly conflicting but well-intentioned objectives?

In [ ]:
messages = [{"role": "user", "content": result}]
copetitor = []
answer = callLLmApi(
    google_api_key,
    "https://generativelanguage.googleapis.com/v1beta/openai/",
    "gemini-2.5-flash-lite",
    messages
)
copetitor.append({"gemini-2.5-flash-lite":answer})


KeyboardInterrupt: 

In [ ]:
openai = OpenAI()
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
question = response.choices[0].message.content
print(question)


In [7]:
competitors = []
answers = []
messages = [{"role": "user", "content": question}]

In [ ]:
# The API we know well

model_name = "gpt-4o-mini"

response = openai.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
# Anthropic has a slightly different API, and Max Tokens is required

model_name = "claude-3-7-sonnet-latest"

claude = Anthropic()
response = claude.messages.create(model=model_name, messages=messages, max_tokens=1000)
answer = response.content[0].text

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
model_name = "deepseek-chat"

response = deepseek.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.3-70b-versatile"

response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)


## For the next cell, we will use Ollama

Ollama runs a local web service that gives an OpenAI compatible endpoint,  
and runs models locally using high performance C++ code.

If you don't have Ollama, install it here by visiting https://ollama.com then pressing Download and following the instructions.

After it's installed, you should be able to visit here: http://localhost:11434 and see the message "Ollama is running"

You might need to restart Cursor (and maybe reboot). Then open a Terminal (control+\`) and run `ollama serve`

Useful Ollama commands (run these in the terminal, or with an exclamation mark in this notebook):

`ollama pull <model_name>` downloads a model locally  
`ollama ls` lists all the models you've downloaded  
`ollama rm <model_name>` deletes the specified model from your downloads

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Super important - ignore me at your peril!</h2>
            <span style="color:#ff7800;">The model called <b>llama3.3</b> is FAR too large for home computers - it's not intended for personal computing and will consume all your resources! Stick with the nicely sized <b>llama3.2</b> or <b>llama3.2:1b</b> and if you want larger, try llama3.1 or smaller variants of Qwen, Gemma, Phi or DeepSeek. See the <A href="https://ollama.com/models">the Ollama models page</a> for a full list of models and sizes.
            </span>
        </td>
    </tr>
</table>

In [ ]:
!ollama pull llama3.2

In [ ]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3.2"

response = ollama.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
# So where are we?

print(competitors)
print(answers)


In [ ]:
# It's nice to know how to use "zip"
for competitor, answer in zip(competitors, answers):
    print(f"Competitor: {competitor}\n\n{answer}")


In [20]:
# Let's bring this together - note the use of "enumerate"

together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

In [ ]:
print(together)

In [22]:
judge = f"""You are judging a competition between {len(competitors)} competitors.
Each model has been given this question:

{question}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


In [ ]:
print(judge)

In [29]:
judge_messages = [{"role": "user", "content": judge}]

In [ ]:
# Judgement time!

openai = OpenAI()
response = openai.chat.completions.create(
    model="o3-mini",
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)


In [ ]:
# OK let's turn this into results!

results_dict = json.loads(results)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = competitors[int(result)-1]
    print(f"Rank {index+1}: {competitor}")

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">Which pattern(s) did this use? Try updating this to add another Agentic design pattern.
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">These kinds of patterns - to send a task to multiple models, and evaluate results,
            are common where you need to improve the quality of your LLM response. This approach can be universally applied
            to business projects where accuracy is critical.
            </span>
        </td>
    </tr>
</table>